# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
pprint(mongo.list_database_names())

['admin', 'classDB', 'config', 'local', 'travel_db', 'uk_food']


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our new database
pprint(db.list_collection_names())

['establishments']


In [19]:
# review a document in the establishments collection
pprint(db.establishments.find_one())


{'AddressLine1': 'The Leas',
 'AddressLine2': 'Kingsdown',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Walmer and Kingsdown Golf Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.793855801259,
 'FHRSID': 452092,
 'LocalAuthorityBusinessID': 'PI/000040614',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT14 8EP',
 'RatingDate': '2019-09-10T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cb18fe8dc1f5042060491a'),
 'geocode': {'latitude': '51.1787300109863', 'longitude': '1.40325689315796'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/452092',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractD

In [12]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
restaurant_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [14]:
# Insert the new restaurant into the collection
establishments.insert_one(restaurant_data)

In [23]:
# Check that the new restaurant was inserted
pprint(db.establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cb2b4cbf28d4d32a090f92'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [33]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID":1, "BusinessType":1}
results = establishments.find_one(query,fields)
pprint(results)
id_results = results["BusinessTypeID"]

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('64cb18fe8dc1f5042060491e')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
update_query = {"BusinessName":"Penang Flavours"}
update_fields = {"$set":{"BusinessTypeID":id_results}}
establishments.update_one(update_query,update_fields)

In [36]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {"BusinessTypeID"}
results = establishments.find_one(query,fields)
pprint(results)

{'BusinessTypeID': 1, '_id': ObjectId('64cb2b4cbf28d4d32a090f92')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [38]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_docs = db.establishments.find_one({"LocalAuthorityName":"Dover"})
pprint(dover_docs)

{'AddressLine1': 'The Leas',
 'AddressLine2': 'Kingsdown',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Walmer and Kingsdown Golf Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.793855801259,
 'FHRSID': 452092,
 'LocalAuthorityBusinessID': 'PI/000040614',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT14 8EP',
 'RatingDate': '2019-09-10T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cb18fe8dc1f5042060491a'),
 'geocode': {'latitude': '51.1787300109863', 'longitude': '1.40325689315796'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/452092',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractD

In [39]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_query = {"LocalAuthorityName":"Dover"}
establishments.delete_many(delete_query)

In [40]:
# Check if any remaining documents include Dover
count_query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(count_query)

0

In [41]:
# Check that other documents remain with 'find_one'
verify = establishments.find_one()
pprint(verify)

{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cb18fe8dc1f50420604c00'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289353',
       

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [42]:
# Change the data type from String to Decimal for longitude and latitude
latlong_query = {}
latlong_update = [{"$set":{"geocode.latitude":{"$toDouble":"$geocode.latitude"},
                   "geocode.longitude":{"$toDouble":"$geocode.longitude"}}}]
establishments.update_many(latlong_query,latlong_update)


Use `update_many` to convert `RatingValue` to integer numbers.

In [43]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [44]:
# Change the data type from String to Integer for RatingValue
rating_query = {}
rating_update = [{"$set":{"RatingValue":{"$toInt":"$RatingValue"}}}]
establishments.update_many(rating_query,rating_update)

In [47]:
# Check that the coordinates and rating value are now numbers
verify_query = {}
verify_fields = {"geocode.latitude":{"$type":"$geocode.latitude"},
         "geocode.longitude":{"$type":"$geocode.longitude"},
         "RatingValue":{"$type":"$RatingValue"}}
results = establishments.find_one(verify_query,verify_fields)
pprint(results)

{'RatingValue': 'int',
 '_id': ObjectId('64cb18fe8dc1f50420604c00'),
 'geocode': {'latitude': 'double', 'longitude': 'double'}}
